In [1]:
import gc
import os
import sys
# import time
from pprint import pprint

# import googlemaps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from scipy.integrate import odeint

main_dir = os.path.abspath(os.pardir)
sys.path.insert(0, main_dir)
from analysis.download_data import jh_git_url, all_files
from covid_forecast.utils import data_prep as dp
from analysis.estimate_sir_params import estimate_sir_params

pd.options.display.max_rows = 999
pd.options.display.max_columns = 100
pd.options.display.precision = 2
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
export_dir = os.path.join(main_dir, "data", "raw"+os.sep)
countries_fname = "countries.csv"
print(main_dir+"\n"+export_dir)
prov_state_vname = "Province/State"
country_region_vname = "Country/Region"

D:\Users\gjnet\code_projects\covid_forecast
D:\Users\gjnet\code_projects\covid_forecast\data\raw\


# Open files as dataframes
Credit to dgrechka for locations_population. Data [source](https://www.kaggle.com/dgrechka/covid19-global-forecasting-locations-population/metadata)

In [3]:
locations_population = pd.read_csv(os.path.join(main_dir, "data", "raw", "locations_population.csv"))
friendly_fnames = [file.replace("time_series_covid19_", "").replace(".csv", "") for file in all_files]
filename_dct = dict(zip(friendly_fnames, all_files))
pprint(filename_dct)

{'confirmed_global': 'time_series_covid19_confirmed_global.csv',
 'deaths_global': 'time_series_covid19_deaths_global.csv',
 'recovered_global': 'time_series_covid19_recovered_global.csv'}


In [ ]:
df_dct = dict()
for file in filename_dct.keys():
    df_dct[file] = pd.read_csv(jh_git_url+filename_dct[file])  # reads csv from github repo
    exec(file+" = df_dct['"+file+"']")  # creates a reference for friendly_names
    print(file, df_dct[file].shape)
    display(df_dct[file].head(), df_dct[file].describe(include="all"), df_dct[file].dtypes)

In [ ]:
confirmed_global.head()

In [ ]:
confirmed_global[country_region_vname].drop_duplicates().to_csv(export_dir+countries_fname, index=False)

In [ ]:
# # Start timer
# start_time = pd.to_datetime("now")
# time_fmt = "%d/%m/%Y %H:%M:%S"
# print("INFO: start time", start_time.strftime(time_fmt))

# # Filling missing province/state where possible
# gmaps = googlemaps.Client(key=os.getenv("gmaps_api_key"))

# match_keys = ["Province/State", "Country/Region", "Lat", "Long"]
# fillna_keys = ["Province/State", "Country/Region"]
# loc_admin_long_vname = "long_name"

# # Start with the first df with missing province/state values
# temp_df = confirmed_global[match_keys][confirmed_global[prov_state_vname].isna()].copy()

# for i in temp_df.index:
#     row = temp_df.loc[i]

#     # Look up an address with reverse geocoding
#     try:
#         target_loc = gmaps.reverse_geocode((row["Lat"], row["Long"]))[0]['address_components']
#     except IndexError:
#         continue
#     admin_ind = dp.find_admin_loc(target_loc)
#     if admin_ind is not None:
#         temp_df.loc[i, prov_state_vname] = target_loc[dp.find_admin_loc(target_loc)][loc_admin_long_vname]
    
#     # Fill missing with country/region
#     temp_df[prov_state_vname] = temp_df[prov_state_vname].fillna(temp_df[country_region_vname])
    
#     # Gmaps anti-throttling tactic
#     time.sleep(1)  # wait 1 second before the next iteration

# # Fillna missing values
# for df in df_dct.keys():
#     df_dct[df][fillna_keys] = confirmed_global[fillna_keys].fillna(temp_df[fillna_keys])

# # Finish time
# finish_time = pd.to_datetime("now")
# elapsed_time_min = round((finish_time-start_time).total_seconds()/60, 2)
# print("INFO: finished. This took", elapsed_time_min, "minutes.")

In [ ]:
# TODO: population data from Rafa's ARIMA and plug this in s_0
# https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide

# TODO: remove head of country row until you find a value > 0 (use numpy) - remove_initia_zeros = np.trim_zeros(data_[variable]).__len__()

# TODO: return beta, and gamma

# TODO: don't return figure

# TODO: SIR: pass prepared dataframes instead of downloading again

In [ ]:
df_dct = dp.bulk_reduce_mem(df_dct)
# population_df = population_df.pipe(dp.reduce_mem)
gc.collect()

# SIR
[Source code](https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/)

In [ ]:
# # Total population, N.
# N = 1000

# # Initial number of infected and recovered individuals, I0 and R0.
# I0, R0 = 1, 0

# # Everyone else, S0, is susceptible to infection initially.
# S0 = N - I0 - R0

# # Contact rate (aka beta), and mean recovery rate aka (gamma) both (in 1/days).
# beta, gamma = 0.2, 1./10

# # A grid of time points (in days)
# days = 160
# t = np.linspace(0, days, days)

# # Initial conditions vector
# y0 = S0, I0, R0
# # Integrate the SIR equations over the time grid, t.
# ret = odeint(dp.deriv, y0, t, args=(N, beta, gamma))
# S, I, R = ret.T

# # Plot the data on three separate curves for S(t), I(t) and R(t)
# fig = plt.figure(facecolor='w')
# ax = fig.add_subplot(111, axisbelow=True)
# ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
# ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
# ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
# ax.set_facecolor('#dddddd')
# ax.set_xlabel('Time /days')
# ax.set_ylabel('Number (1000s)')
# ax.set_ylim(0,1.2)
# ax.yaxis.set_tick_params(length=0)
# ax.xaxis.set_tick_params(length=0)
# ax.grid(b=True, which='major', c='w', lw=2, ls='-')
# legend = ax.legend()
# legend.get_frame().set_alpha(0.5)
# for spine in ('top', 'right', 'bottom', 'left'):
#     ax.spines[spine].set_visible(False)
# plt.show()

## Estimating SIR beta and gamma parameters

In [ ]:
from analysis.estimate_sir_params import estimate_sir_params
estimate_sir_params(countries=confirmed_global[country_region_vname].drop_duplicates())  # .iloc[:10]